# A 2025 Implementation of Jeremy Howard's " A hacker's guide to Language Models"

The original implementation of this can be found [here](https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb). Since the progress in Language models has been so rapid, some of the models and artificats used in the original notebook have changed, been tweaked or updated for example the LLAMA models, so here i will be trying to use the latest state of the art techniques , models etc.

#### What is a language model ?

I previously defined and implemented a simple language model [here](https://nbsanity.com/static/88f4b8caa233fa6d0a5e5114810403b3/symptom-disease-ulmfit_lightning.html)

In [1]:
#|include: false 
#| code-fold: true
#| output: false
#| code-summary: "Library Installation"

%pip install --upgrade openai
%pip install claudette
%pip install python-dotenv
%pip install -U bitsandbytes
%pip install optimum
%pip install auto-gptq
%pip install Wikipedia-API
%pip install tiktoken
%pip install sentence-transformers
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
#|include: false 
#| code-fold: true
#| output: false
#| code-summary: "Library Import"

import tokenize, ast
from io import BytesIO
import os

from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
import torch
import torch

import ipywidgets as widgets

from openai import OpenAI

In [3]:
#|include: false 
#| code-fold: true
#| output: false
#| code-summary: "Library Import"
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


## Base Models

### Tokens

Our language models process text using tokens and always break down text into tokens, which can be words, subwords, etc. This creates a vocabulary of unique tokens, where each token is assigned a unique ID.

With this, all the text passed to a language model can be broken down into tokens. As a model is being trained, it constantly adds new tokens to its vocabulary.

Language models also analyze and learn the semantic relationship between the tokens such as how they are used, or used together. This enables them to predict the most likely token in a sequence of tokens based on the input sequence. This is represented to us as predicting the next word/subword/character in a sentence.

We can also represent the relationships between our tokens with something called embeddings.

In [4]:
from tiktoken import encoding_for_model
enc = encoding_for_model("text-davinci-003")
toks = enc.encode("They are splashing")
toks

[2990, 389, 4328, 2140]

In [15]:
[enc.decode_single_token_bytes(o).decode('utf-8') for o in toks]

['They', ' are', ' spl', 'ashing']

To illustrate, consider the above text, "They are splashing" which has been broken down into [2990, 389, 4328, 2140]. OpenAI currently has a [tool](https://platform.openai.com/tokenizer) we can use to visualize how text is tokenized.

This is what we used above through [tiktoken](https://github.com/openai/tiktoken) to programmatically interact with our API for tokenizing text.

In [16]:
enc_a = encoding_for_model("gpt-3.5-turbo")
toks_a = enc_a.encode("They are splashing")
toks_a

[7009, 527, 12786, 19587]

In [17]:
enc_b = encoding_for_model("o1")
toks_b = enc_b.encode("They are splashing")
toks_b

[12280, 553, 15885, 33306]

As you can see, different models encode our text in different ways. You can see the models we have access to through tiktoken [here](https://github.com/openai/tiktoken/blob/main/tiktoken/model.py).

When you train your own model, you create your own vocab.

In [18]:
[enc_b.decode_single_token_bytes(o).decode('utf-8') for o in toks_b]

['They', ' are', ' spl', 'ashing']

In [19]:
[enc.decode_single_token_bytes(o).decode('utf-8') for o in toks]

['They', ' are', ' spl', 'ashing']

## OpenAI API

In [33]:
# Create a password widget for secure input
api_key_input = widgets.Password(
    description='OpenAI API Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

display(api_key_input)

# In a separate cell, after the user inputs their key:
os.environ["OPENAI_API_KEY"] = api_key_input.value
client = OpenAI()

Password(description='OpenAI API Key:', layout=Layout(width='50%'), style=TextStyle(description_width='initial…

In [31]:
#client = OpenAI()

response = client.responses.create(
  model="gpt-4o",
  input="Tell me a three sentence bedtime story about a unicorn."
)

print(response)


AttributeError: 'Anthropic' object has no attribute 'responses'

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "developer", "content": "Talk like a pirate."},
        {
            "role": "user",
            "content": "How do I check if a Python object is an instance of a class?",
        },
    ],
)

print(completion.choices[0].message.content)

## Claude API

In [32]:
import ipywidgets as widgets
import os
from anthropic import Anthropic

# Create a password widget for secure input
api_key_input = widgets.Password(
    description='Anthropic API Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

display(api_key_input)

# In a separate cell, after the user inputs their key:
os.environ["ANTHROPIC_API_KEY"] = api_key_input.value
client = Anthropic()

Password(description='Anthropic API Key:', layout=Layout(width='50%'), style=TextStyle(description_width='init…

In [22]:
from claudette import *

In [23]:
import anthropic

In [25]:
models

['claude-3-opus-20240229',
 'claude-3-7-sonnet-20250219',
 'claude-3-5-sonnet-20241022',
 'claude-3-haiku-20240307',
 'claude-3-5-haiku-20241022']

In [26]:
model = models[1] #selects 'claude-3-7-sonnet-20250219'


In [27]:

# Option 1: Create a Claudette Client first, then pass it to Chat
#c = Client(model, cli=client)  # Pass your Anthropic client here
#chat = Chat(cli=c, sp="You are a helpful and concise assistant.")

# Option 2: Or directly when creating the Chat
chat = Chat(model=model, cli=Client(model, cli=client), sp="You are a helpful and concise assistant.")

# Now you can use the chat
chat("I'm Silver Rubanza")

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}

## Custom Models 

In [2]:
mn = "meta-llama/Llama-2-7b-hf"

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)

In [4]:
#model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Silver Rubanza is a "
toks = tokr(prompt, return_tensors="pt")

In [18]:
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors="pt")

In [19]:
toks

{'input_ids': tensor([[    1,  5677,  6764, 17430,   338,   263, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [20]:
tokr.batch_decode(toks['input_ids'])

['<s> Jeremy Howard is a ']

In [21]:
%%time
res = model.generate(**toks.to("cuda"),max_new_tokens=15).to('cpu')
res

CPU times: user 520 ms, sys: 0 ns, total: 520 ms
Wall time: 531 ms


tensor([[    1,  5677,  6764, 17430,   338,   263, 29871, 29941, 29945, 29899,
          6360, 29899,  1025,   767,   515,   278,  3303,  3900,  1058,   471,
         24383,   297]])

In [22]:
tokr.batch_decode(res)

['<s> Jeremy Howard is a 35-year-old man from the United States who was arrested in']

In [23]:
#model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(mn,device_map=0,torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16)

Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_p

In [25]:
%%time
res = model.generate(**toks.to("cuda"),max_new_tokens=15).to('cpu')
res

CPU times: user 498 ms, sys: 6.15 ms, total: 504 ms
Wall time: 562 ms


tensor([[    1,  5677,  6764, 17430,   338,   263, 29871, 29941, 29945, 29899,
          6360, 29899,  1025,   767,   515,   278,  3303,  3900,  1058,   471,
         24383,   297]])

In [26]:
tokr.batch_decode(res)

['<s> Jeremy Howard is a 35-year-old man from the United States who was arrested in']

In [27]:
def gen(p, maxlen=15, sample=True):
  toks = tokr(p,return_tensors="pt")
  res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
  return tokr.batch_decode(res)

In [28]:
%%time
gen(prompt,50)

CPU times: user 1.69 s, sys: 2.83 ms, total: 1.69 s
Wall time: 1.72 s


['<s> Jeremy Howard is a 25-year-old man from a small town in the Midwest. He has been playing music for most of his life and has always been interested in the sounds of the past. As a teenager, he became obsessed with the']

#### Llama 3.1

In [16]:
import transformers
import torch

model_id = "meta-llama/Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

pipeline("Silver Rubanza is a ")


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Silver Rubanza is a 12x12" acrylic painting on canvas. It is painted with a palette knife and is textured and'}]

In [17]:
pipeline("Jeremy Howards is a")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Jeremy Howards is a 2004 graduate of the University of Georgia and is currently an associate attorney with the firm. Jeremy'}]

## Other Models

### Stable Beluga

In [ ]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [ ]:
sb_sys = "### System:\n You are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can. . \n\n"

In [ ]:
def mk_prompt(user, syst=sb_sys):
    return f"{syst} ### User: {user}\n\n### Assistant:\n"

In [ ]:
ques = "Who is Silver Rubanza?"

In [ ]:
%%time
gen(mk_prompt(ques), 150)

### Open Orca / Playtpus 2

In [ ]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
def mk_oo_prompt(user):
    return f"### Instruction: {user}\n\n ### Response:\n"

In [ ]:
gen(mk_oo_prompt(ques),150)

## Retrival Augmented Generation

In [4]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
from wikipediaapi import Wikipedia

In [ ]:
wiki = Wikipedia('JeremyHowardBot/0.0', 'en')
jh_page = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_page = jh_page.split('\nReferences\n')[0]

In [ ]:
jh_page

In [ ]:
print(jh_page[:500])

In [ ]:
print(type(jh_page))

In [ ]:
sr_page = 'Hi, I am Rubanza Silver A coder with a background in Software Engineering My work and interest lie in working on various steps of the machine learning lifecycle from Exploratory Data Analysis, Data wrangling, Feature Engineering to Model building, deployment, testing, monitoring, etc.I integrate machine learning models into general software solutions, all in the context of solving a given problem. Likewise, I am proficient with Python, PyTorch, and many other libraries such as fastai, sklearn, etc.I also have over 6 years of experience developing software applications using HTML, CSS, Javascript, React JS, etc.Below are examples of my work'

In [ ]:
len(jh_page.split()),len(sr_page.split())

In [ ]:
ques_ctx = f"""Answer the question with the help of the provided context.

## Context
{sr_page}

## Question
{ques}

"""

In [ ]:
res = gen(mk_prompt(ques_ctx),300)

In [ ]:
print(res[0].split('### Assistant:\n')[1])

In [ ]:
%pip install sentence-transformers
%pip install PyPDF2

In [5]:
from sentence_transformers import SentenceTransformer

In [ ]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

In [ ]:
jh = jh_page.split('\n\n')[0]
print(jh)

In [ ]:
sr_page

In [ ]:
sr = sr_page.split('\n\n')[0]
print(sr)

In [ ]:
q_emb,jh_emb,sr_emb = emb_model.encode([ques,jh,sr], convert_to_tensor=True)

In [ ]:
import torch.nn.functional as F

In [ ]:
F.cosine_similarity(q_emb, jh_emb, dim=0)

In [ ]:
F.cosine_similarity(q_emb, sr_emb, dim=0)

Ug guidelines

In [6]:
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import PyPDF2
import numpy as np
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text() + "\n\n"
    return text

# Split text into chunks
def split_into_chunks(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

# Setup everything once
def setup_rag_system(pdf_path="ug_cg_23.pdf"):
    # Load PDF
    guidelines_text = extract_text_from_pdf(pdf_path)
    chunks = split_into_chunks(guidelines_text)
    
    # Create embeddings
    emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device="cuda" if torch.cuda.is_available() else "cpu")
    
    # Cache embeddings
    chunk_embeddings = []
    for chunk in chunks:
        embedding = emb_model.encode(chunk, convert_to_tensor=True)
        chunk_embeddings.append(embedding)
    
    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        "TheBloke/Llama-2-7b-Chat-GPTQ", 
        device_map=0, 
        torch_dtype=torch.float16
    )
    tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7b-Chat-GPTQ")
    
    return {
        "chunks": chunks,
        "chunk_embeddings": chunk_embeddings,
        "model": model,
        "tokenizer": tokenizer,
        "emb_model": emb_model
    }

# Retrieve context based on symptoms
def retrieve_context(query, rag_system, top_k=3):
    emb_model = rag_system["emb_model"]
    chunks = rag_system["chunks"]
    chunk_embeddings = rag_system["chunk_embeddings"]
    
    # Encode query
    query_embedding = emb_model.encode(query, convert_to_tensor=True)
    
    # Calculate similarities
    similarities = []
    for chunk_emb in chunk_embeddings:
        similarity = F.cosine_similarity(query_embedding, chunk_emb, dim=0)
        similarities.append(similarity.item())
    
    # Get top k chunks
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [(chunks[i], similarities[i]) for i in top_indices]

# Generate prompt
def mk_oo_prompt(user):
    return f"### Instruction: {user}\n\n### Response:\n"

# Generate response using the model
def gen(p, model, tokenizer, maxlen=150, sample=True):
    toks = tokenizer(p, return_tensors="pt")
    res = model.generate(
        **toks.to("cuda"), 
        max_new_tokens=maxlen, 
        do_sample=sample
    ).to('cpu')
    return tokenizer.batch_decode(res)

# Main function to diagnose based on symptoms
def diagnose_symptoms(symptoms_text, rag_system, top_k=3):
    # Retrieve relevant context
    relevant_chunks = retrieve_context(symptoms_text, rag_system, top_k)
    
    # Format context
    context = "\n\n".join([chunk for chunk, _ in relevant_chunks])
    
    # Create prompt
    prompt = mk_oo_prompt(f"""Answer the question with the help of the provided context.

## Context

{context}

## Question

Based on these symptoms: "{symptoms_text}", what are the possible diseases according to Ugandan Clinical Guidelines?""")
    
    # Generate response
    model = rag_system["model"]
    tokenizer = rag_system["tokenizer"]
    response = gen(prompt, model, tokenizer, maxlen=300)[0]
    
    # Extract assistant's response
    answer = response.split("### Response:\n")[-1]
    
    result = {
        "diagnosis": answer,
        "context_used": [chunk for chunk, _ in relevant_chunks],
        "similarity_scores": [sim for chunk, sim in relevant_chunks]
    }
    
    # Print results to console
    print("\n=== Diagnosis ===")
    print(result["diagnosis"])
    
    print("\n=== Top Relevant Guidelines Used ===")
    for i, (context, score) in enumerate(zip(result["context_used"], result["similarity_scores"]), 1):
        print(f"--- Context {i} (Similarity: {score:.3f}) ---")
        print(context[:200] + "..." if len(context) > 200 else context)
        print()
        
    return result

# Example usage - set up once and reuse
rag_system = setup_rag_system()

# When you want to diagnose symptoms, just call this function
result = diagnose_symptoms("Ear pain, fever, and difficulty hearing", rag_system)
# When you want to diagnose symptoms, just call this function


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', '


=== Diagnosis ===
Please see 

## nose of foreign object
## 

This is not

-r


How foreign body or tissue in ear, nose congestion 
## 


 

##
Ug




# foreign body's

Clinical diagnosis

The nasto

# nasto
##



and ear examin





 nose


Tn
Infection



infection

the 






and ear

##


the






ear



Clin









ears









##







































































































































=== Top Relevant Guidelines Used ===
--- Context 1 (Similarity: 0.762) ---
years Causes ~ Bacterial infection, e.g., Streptococcus pneumoniae, Haemoph - ilus influenzae ~ Commonly follows an acute infection of the upper respiratory tract Clinical features ~ Acute onset of pa...

--- Context 2 (Similarity: 0.756) ---
(Otitis Media with Effusion) ............................................................. 953 21.1.6 Mastoiditis .........................................................................................


In [5]:
result = diagnose_symptoms("Ive been having pain and swelling in my knees and wrists, especially in the morning. It gets a little better as the day goes on. I'm also feeling fatigued and have noticed a rash on my face", rag_system)


=== Diagnosis ===
........ ........ ... ................ ................ Subject  ------------ ........ ........ ... .. ........ ................ ........ .... ................ ................ ................ ... ........ ------- ... ................ ................ ==== ..... ................ ........  ................  ... ................   ................ ........ ................ ................ ........    ................   ... ........ ................ ........  ................ ................ ................  ................ ................ ................  ................  ................ ................    ................ ........     ................       ...  ...     ........ ........  ...  ................                   ........    ........         ................          ........................     ..   ................ ................     ................  ..."    ................ ................ ................ ...  ................   ......

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import PyPDF2
import numpy as np
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text() + "\n\n"
    return text

def split_into_chunks(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

def setup_rag_system(pdf_path="/teamspace/studios/this_studio/ug_cg_23.pdf"):
    # Load PDF
    guidelines_text = extract_text_from_pdf(pdf_path)
    chunks = split_into_chunks(guidelines_text)
    
    # Create embeddings
    emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device="cuda" if torch.cuda.is_available() else "cpu")
    
    # Cache embeddings
    chunk_embeddings = []
    for chunk in chunks:
        embedding = emb_model.encode(chunk, convert_to_tensor=True)
        chunk_embeddings.append(embedding)
    
    # Keep using Llama as requested
    model = AutoModelForCausalLM.from_pretrained(
        "TheBloke/Llama-2-7b-Chat-GPTQ", 
        device_map=0, 
        torch_dtype=torch.float16
    )
    tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7b-Chat-GPTQ")
    
    return {
        "chunks": chunks,
        "chunk_embeddings": chunk_embeddings,
        "model": model,
        "tokenizer": tokenizer,
        "emb_model": emb_model
    }

def retrieve_context(query, rag_system, top_k=3):
    emb_model = rag_system["emb_model"]
    chunks = rag_system["chunks"]
    chunk_embeddings = rag_system["chunk_embeddings"]
    
    # Encode query
    query_embedding = emb_model.encode(query, convert_to_tensor=True)
    
    # Calculate similarities
    similarities = []
    for chunk_emb in chunk_embeddings:
        similarity = F.cosine_similarity(query_embedding, chunk_emb, dim=0)
        similarities.append(similarity.item())
    
    # Get top k chunks
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [(chunks[i], similarities[i]) for i in top_indices]

# Fixed: Better prompt format for Llama-2-chat
def create_llama_prompt(context, symptoms):
    return f"""<s>[INST] <<SYS>>
You are a medical diagnostic assistant specialized in Ugandan healthcare guidelines.
Your task is to provide a differential diagnosis based on the symptoms described.
Be specific, concise, and refer to the Ugandan Clinical Guidelines in your response.
<</SYS>>

Based on the following excerpts from the Ugandan Clinical Guidelines:

{context}

Provide a differential diagnosis for a patient with these symptoms: {symptoms}

List the top 3-5 possible diagnoses in order of likelihood, with a brief explanation for each. [/INST]"""

def gen(prompt, model, tokenizer, max_length=500):
    # Fixed: Better token generation parameters
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        do_sample=True,
        temperature=0.4,  # Lower temperature for more focused responses
        top_p=0.9,
        repetition_penalty=1.2,  # Avoid repetitive patterns like "............"
        num_beams=3  # Using beam search for better coherence
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def diagnose_symptoms(symptoms_text, rag_system, top_k=3):
    # Retrieve relevant context
    relevant_chunks = retrieve_context(symptoms_text, rag_system, top_k)
    
    # Format context - improved to be more structured
    context_texts = []
    for i, (chunk, score) in enumerate(relevant_chunks, 1):
        # Keep only the most meaningful text, removing excessive punctuation
        clean_chunk = ' '.join([s for s in chunk.split() if not s.startswith('.')])
        context_texts.append(f"SECTION {i}:\n{clean_chunk}")
    
    context = "\n\n".join(context_texts)
    
    # Create prompt with the fixed format
    prompt = create_llama_prompt(context, symptoms_text)
    
    # Generate response
    model = rag_system["model"]
    tokenizer = rag_system["tokenizer"]
    full_response = gen(prompt, model, tokenizer)
    
    # Extract just the model's response
    response_parts = full_response.split("[/INST]")
    if len(response_parts) > 1:
        response = response_parts[1].strip()
    else:
        response = full_response
    
    result = {
        "diagnosis": response,
        "context_used": [chunk for chunk, _ in relevant_chunks],
        "similarity_scores": [sim for chunk, sim in relevant_chunks]
    }
    
    # Print results 
    print("\n=== Diagnosis ===")
    print(result["diagnosis"])
    
    print("\n=== Top Relevant Guidelines Used ===")
    for i, (context, score) in enumerate(zip(result["context_used"], result["similarity_scores"]), 1):
        print(f"--- Context {i} (Similarity: {score:.3f}) ---")
        print(context[:200] + "..." if len(context) > 200 else context)
        print()
        
    return result

# Example usage with the fixed path
rag_system = setup_rag_system()
#result = diagnose_symptoms("Ear pain, fever, and difficulty hearing", rag_system)
result = diagnose_symptoms("Ive been having pain and swelling in my knees and wrists, especially in the morning. It gets a little better as the day goes on. I'm also feeling fatigued and have noticed a rash on my face.", rag_system)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', '


=== Diagnosis ===


=== Top Relevant Guidelines Used ===
--- Context 1 (Similarity: 0.713) ---
(Otitis Media with Effusion) ............................................................. 953 21.1.6 Mastoiditis .........................................................................................

--- Context 2 (Similarity: 0.706) ---
Clinical Features and Investigations ............................................................... 623 13.1.2 Nociceptive Pain Management ...............................................................

--- Context 3 (Similarity: 0.687) ---
Hormonal changes ~ Infections: Spirochaetes, Bacilli, Candida, Measles virus, Herpes simplex virus 1023 Uganda Clinical Guidelines 2023 CHAPTER 23 : Oral and Dental ConditionsClinical features ~ Infla...



In [6]:
result = diagnose_symptoms("Ive been having pain and swelling in my knees and wrists, especially in the morning. It gets a little better as the day goes on. I'm also feeling fatigued and have noticed a rash on my face.", rag_system)


=== Diagnosis ===


=== Top Relevant Guidelines Used ===
--- Context 1 (Similarity: 0.713) ---
(Otitis Media with Effusion) ............................................................. 953 21.1.6 Mastoiditis .........................................................................................

--- Context 2 (Similarity: 0.706) ---
Clinical Features and Investigations ............................................................... 623 13.1.2 Nociceptive Pain Management ...............................................................

--- Context 3 (Similarity: 0.687) ---
Hormonal changes ~ Infections: Spirochaetes, Bacilli, Candida, Measles virus, Herpes simplex virus 1023 Uganda Clinical Guidelines 2023 CHAPTER 23 : Oral and Dental ConditionsClinical features ~ Infla...



In [5]:
result = diagnose_symptoms("Ear pain, fever, and difficulty hearing", rag_system)


=== Diagnosis ===
Based on the symptoms provided, here are the top 3-5 possible diagnoses for the patient, in order of likelihood:
1. Acute Otitis Media (AOM): The patient's symptoms of ear pain, fever, and difficulty hearing are classic signs of AOM, a common bacterial infection of the middle ear. AOM is the most common cause of ear infections in children and can also occur in adults.
2. Otitis Media with Effusion (OME): OME is a condition where there is fluid buildup in the middle ear without an infection. The patient's symptoms of ear pain, fever, and difficulty hearing could be due to OME.
3. Mastoiditis: Mastoiditis is an infection of the mastoid bone behind the ear, which can occur as a complication of untreated AOM or OME. The patient's severe ear pain and swelling in the post-auricular area suggest the possibility of mastoiditis.
4. Temporomandibular Joint (TMJ) Disorder: The patient's ear pain and difficulty opening the jaw could be related to a TMJ disorder, which can cause 

In [8]:
d1 = "I’ve been having pain and swelling in my knees and wrists, especially in the morning. It gets a little better as the day goes on. I'm also feeling fatigued and have noticed a rash on my face."

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import PyPDF2
import numpy as np
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text() + "\n\n"
    return text

def split_into_chunks(text, chunk_size=500, overlap=50):
    # Clean text first - remove excessive punctuation and page numbers
    text = re.sub(r'\.{3,}', ' ', text)  # Replace sequences of dots with space
    text = re.sub(r'\d{3,4}\s+Uganda\s+Clinical\s+Guidelines\s+\d{4}', '', text)  # Remove page markers
    
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

def setup_rag_system(pdf_path="/teamspace/studios/this_studio/ug_cg_23.pdf"):
    # Load PDF
    guidelines_text = extract_text_from_pdf(pdf_path)
    chunks = split_into_chunks(guidelines_text)
    
    # Create embeddings
    emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device="cuda" if torch.cuda.is_available() else "cpu")
    
    # Cache embeddings
    chunk_embeddings = []
    for chunk in chunks:
        embedding = emb_model.encode(chunk, convert_to_tensor=True)
        chunk_embeddings.append(embedding)
    
    model = AutoModelForCausalLM.from_pretrained(
        "TheBloke/Llama-2-7b-Chat-GPTQ", 
        device_map=0, 
        torch_dtype=torch.float16
    )
    tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7b-Chat-GPTQ")
    
    return {
        "chunks": chunks,
        "chunk_embeddings": chunk_embeddings,
        "model": model,
        "tokenizer": tokenizer,
        "emb_model": emb_model
    }

def retrieve_context(query, rag_system, top_k=5):  # Increased from 3 to 5
    emb_model = rag_system["emb_model"]
    chunks = rag_system["chunks"]
    chunk_embeddings = rag_system["chunk_embeddings"]
    
    # Add medical keywords to enhance the query
    symptoms = query.lower()
    enhanced_query = query
    if "joint" in symptoms or "knee" in symptoms or "wrist" in symptoms:
        enhanced_query += " arthritis rheumatoid joint inflammation"
    elif "ear" in symptoms:
        enhanced_query += " otitis media ear infection"
    elif "skin" in symptoms or "rash" in symptoms:
        enhanced_query += " dermatitis skin condition eczema"
    
    # Encode query
    query_embedding = emb_model.encode(enhanced_query, convert_to_tensor=True)
    
    # Calculate similarities
    similarities = []
    for chunk_emb in chunk_embeddings:
        similarity = F.cosine_similarity(query_embedding, chunk_emb, dim=0)
        similarities.append(similarity.item())
    
    # Get top k chunks
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [(chunks[i], similarities[i]) for i in top_indices]

def create_llama_prompt(context, symptoms):
    return f"""<s>[INST] <<SYS>>
You are a medical diagnostic assistant following Ugandan healthcare guidelines.
Your task is to provide a differential diagnosis based on patient symptoms.
Be specific, concise, and thorough in your analysis.
If the symptoms don't match perfectly with the guidelines, still provide your best medical assessment.
<</SYS>>

Based on the following excerpts from the Ugandan Clinical Guidelines and your medical knowledge:

{context}

Provide a differential diagnosis for a patient presenting with these symptoms: {symptoms}

List the top 3-5 possible diagnoses in order of likelihood, with a brief explanation for each.
Even if the guidelines don't directly address these symptoms, use your medical knowledge to make appropriate inferences.
[/INST]"""

def gen(prompt, model, tokenizer, max_length=500):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        do_sample=True,
        temperature=0.3,  # Lowered further for more focused responses
        top_p=0.85,
        repetition_penalty=1.3,  # Increased to further avoid repetition
        length_penalty=1.0,  # Encourages complete sentences
        no_repeat_ngram_size=3  # Prevent repeating 3-grams
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def diagnose_symptoms(symptoms_text, rag_system, top_k=5):
    # Retrieve relevant context
    relevant_chunks = retrieve_context(symptoms_text, rag_system, top_k)
    
    # Format context - improved to be more structured and cleaner
    context_texts = []
    for i, (chunk, score) in enumerate(relevant_chunks, 1):
        # Remove page numbers and excessive punctuation
        clean_chunk = re.sub(r'\.{3,}', ' ', chunk)  # Replace sequences of dots with space
        clean_chunk = re.sub(r'\d{3,4}\s+Uganda\s+Clinical\s+Guidelines\s+\d{4}', '', clean_chunk)
        clean_chunk = re.sub(r'\s{2,}', ' ', clean_chunk)  # Replace multiple spaces with single space
        
        # Extract any disease names or chapter titles for better context
        disease_match = re.search(r'(CHAPTER \d+[^\.]+|[A-Z][a-z]+\s+[A-Z][a-z]+\s+(Disease|Syndrome|Disorder|Infection|Pain))', clean_chunk)
        section_title = f" - {disease_match.group(0)}" if disease_match else ""
        
        context_texts.append(f"SECTION {i}{section_title}:\n{clean_chunk[:500]}")  # Limit length to keep focused
    
    context = "\n\n".join(context_texts)
    
    # Create prompt with the improved format
    prompt = create_llama_prompt(context, symptoms_text)
    
    # Generate response
    model = rag_system["model"]
    tokenizer = rag_system["tokenizer"]
    full_response = gen(prompt, model, tokenizer, max_length=800)  # Increased length for more complete responses
    
    # Extract just the model's response
    response_parts = full_response.split("[/INST]")
    if len(response_parts) > 1:
        response = response_parts[1].strip()
    else:
        response = full_response
    
    result = {
        "diagnosis": response,
        "context_used": [chunk for chunk, _ in relevant_chunks],
        "similarity_scores": [sim for chunk, sim in relevant_chunks]
    }
    
    # Print results 
    print("\n=== Diagnosis ===")
    print(result["diagnosis"])
    
    print("\n=== Top Relevant Guidelines Used ===")
    for i, (context, score) in enumerate(zip(result["context_used"], result["similarity_scores"]), 1):
        # Print cleaner context summaries
        clean_summary = re.sub(r'\.{3,}', ' ', context[:200])
        print(f"--- Context {i} (Similarity: {score:.3f}) ---")
        print(clean_summary + "..." if len(context) > 200 else clean_summary)
        print()
        
    return result

# Usage remains the same
rag_system = setup_rag_system()

Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_p

In [8]:
# First, set up the RAG system (you only need to do this once)
rag_system = setup_rag_system()

# Then use diagnose_symptoms with your prompt
result = diagnose_symptoms("Ive been having pain and swelling in my knees and wrists, especially in the morning. It gets a little better as the day goes on. I'm also feeling fatigued and have noticed a rash on my face.", rag_system)

Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_p


=== Diagnosis ===
Based on the provided information, here are five potential diagnoses along with their probabilities;
Reactive Arthritispotentially 90%
This condition occurs when one experiences inflammation due to another underlying cause such as an autoimmune response or infection. Given the presence of both knee and w elbow pain accompanied by fatigue and facial rash, reactive artrh itis seems likely. However further testing would be required t confirm this diagnosis.
OsteoArthritishigh possibility 85%
As a chronic degenerative condition characterized by cartilage breakdown within the joints leading to friction and subsequent pain, osteoathritis could explain the persistent knee soreness experienced by our patient. This diagnosis might also account for the early morning stiffness reported since the body typically feels more rigid during periods of inactivity overnight. While we cannot rule out alternative causes entirely without additional examination results ,it remains plausable

In [9]:
# First, set up the RAG system (you only need to do this once)
rag_system = setup_rag_system()

# Then use diagnose_symptoms with your prompt
result = diagnose_symptoms("Ear pain, fever, and difficulty hearing", rag_system)

Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_p


=== Diagnosis ===
Based on the provided information, here are my top 5 differential diagnoses for this patient:
1. Acute Otitis Media (AOM): The most likely diagnosis given the patient's symptomology of ear pain, fee, and difficult y hearing. This common pediatric illness occurs when the middle ears become inflamed due to an infection. According to the clinical presentation, it seems like the infection might be caused by group B beta-hemolytic streptococi.
2. Ototoxicity: As a result of taking certain medications or exposure to loud noises, this condition can damage the inner ear and interfere with balance and hearing. Given the patient 's age, it would be important to determine whether they have been exposed to anything harmful recently.
3. Meniere’s disease: An inner ear disorder characterized by episodes of vertigo, nausea, vomiting, abnormal hearing loss, and tinnunculus(ringing in the ears). Although uncommon in youngsters, menieres disease could also contribute to difficulties w

## Finetuning

import datasets

In [ ]:
ds = datasets.load_dataset('knowrohit07/know_medical_dialogue_v2')

In [ ]:
trn = ds['train']
trn[3]

In [ ]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst

In [ ]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT:"""

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])

## References

[Understanding Tokenization](https://learn.microsoft.com/en-us/dotnet/ai/conceptual/understanding-tokens)

[Hacker's guide to Language Models by Jeremy Howard](https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb)